In [1]:
import requests
import pandas as pd
from io import StringIO


# Список источников, нам понадобится информация по 5 видам сырья:
# алюминий, медь, свинец, никель и цинк
metalls = {
    "aluminium": "Al",
    "copper": "Cu",
    "lead": "Pb",
    "nickel": "Ni",
    "zink": "Zn",
}


def get_metalls(metall_name: str):

    # Запрос к источнику
    url = f"https://www.westmetall.com/en/markdaten.php?action=table&field=LME_{metalls[metall_name]}_cash"

    responce = requests.get(url)
    # Чтение ответа
    responce_data = pd.read_html(StringIO(responce.text))

    data = pd.DataFrame()
    for i in range(len(responce_data)):
        data = pd.concat([data, responce_data[i]])

    # Предобработка
    responce_cur_data = data.copy()
    responce_cur_data = responce_cur_data.loc[responce_cur_data["date"] != "date"]

    # Преобразование в формат datetime
    responce_cur_data["date"] = responce_cur_data["date"].apply(pd.to_datetime).dt.date

    columns = responce_cur_data.columns.to_list()[1:]

    # сброс пропусков
    for column_name in columns:
        responce_cur_data[column_name] = responce_cur_data[column_name].str.replace(
            "-", "0"
        )

    # Преобразование в числовой формат
    for column_name in columns:
        responce_cur_data[column_name] = responce_cur_data[column_name].apply(
            pd.to_numeric
        )

    responce_cur_data = responce_cur_data[responce_cur_data.iloc[:, 1] != 0]

    for column_name in columns:
        responce_cur_data[column_name] = responce_cur_data[column_name].astype(
            "float64"
        )

    with open(f'./data/{metall_name}.csv', 'w') as file:
        responce_cur_data.to_csv(file, index=False)
        


if __name__ == "__main__":

    for metall in metalls.keys():
        # Получение данных
        data = get_metalls(metall)